In [1]:
import glob 
import numpy as np 
import os 
import shutil 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score,recall_score
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator,  load_img, img_to_array, array_to_img 
np.random.seed(42) 
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model 
import keras 
import matplotlib.pyplot as plt 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential 
from keras import optimizers  

In [2]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))        

In [3]:
IM_SHAPE=(225,225)
input_shape=(IM_SHAPE[0],IM_SHAPE[1],3)

In [4]:
batch_size = 32
num_classes = 2 
epochs = 32
train_datagen = ImageDataGenerator(rescale=1./255,
                                    preprocessing_function=preprocess_input,
                                      validation_split=0.3)
val_datagen=ImageDataGenerator(rescale=1./255,
                               preprocessing_function=preprocess_input,
                                      validation_split=0.2)

# train_generator = train_datagen.flow_from_directory(
#     '../input/covidog/dataset',
#     target_size=IM_SHAPE,

#     batch_size=batch_size,
#     class_mode='categorical') # set as training data

train_generator_NEW = train_datagen.flow_from_directory(
    '/kaggle/input/covidnew/ALL/Train', # same directory as training data
    target_size=IM_SHAPE,
    subset='training',
    batch_size=batch_size,
    class_mode='categorical') # set as validation data
validation_generator_NEW= train_datagen.flow_from_directory(
    '/kaggle/input/covidnew/ALL/Train',
#     batch_size=16,
    subset='validation',
    class_mode='categorical',
    target_size=IM_SHAPE
)
test_generator_NEW=val_datagen.flow_from_directory(
    '/kaggle/input/covidnew/ALL/Test',
    #batch_size=16,
    class_mode='categorical',
    target_size=IM_SHAPE
)

Found 14461 images belonging to 2 classes.
Found 6196 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
#from keras.applications import DenseNet121


vgg = tf.keras.applications.DenseNet121(include_top=False, weights='imagenet',  
                                     input_shape=input_shape) 

# we have removed the final part of the classifier pertaining to the VGG-16 model 
#since we will be building our own classifier and leveraging VGG as a feature extractor
output = vgg.layers[-1].output 
output = keras.layers.Flatten()(output) 
vgg_model = Model(vgg.input, output) 
vgg_model.trainable = False 
 
for layer in vgg_model.layers: 
    layer.trainable = False 

#vgg_model.summary()

2023-02-08 02:43:41.294977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 02:43:41.384748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 02:43:41.385595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 02:43:41.387499: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

29097984/29084464 [==============================] - 2s 0us/step


In [6]:
# To verify that the layers of the VGG-16 model are frozen, we can use the following code:
import pandas as pd 
pd.set_option('max_colwidth', -1) 
 
layers = [(layer, layer.name, layer.trainable) for layer in 
           vgg_model.layers] 
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x7f4cc8bec250>,input_1,False
1,<keras.layers.convolutional.ZeroPadding2D object at 0x7f4cc7407fd0>,zero_padding2d,False
2,<keras.layers.convolutional.Conv2D object at 0x7f4cc7407390>,conv1/conv,False
3,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f4cc81cad10>,conv1/bn,False
4,<keras.layers.core.Activation object at 0x7f4cc81dd210>,conv1/relu,False
...,...,...,...
423,<keras.layers.convolutional.Conv2D object at 0x7f4cc6f13f90>,conv5_block16_2_conv,False
424,<keras.layers.merge.Concatenate object at 0x7f4cc6ebaa50>,conv5_block16_concat,False
425,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f4cc70f58d0>,bn,False
426,<keras.layers.core.Activation object at 0x7f4cc6ec9bd0>,relu,False


In [11]:
model_TF = Sequential() 

model_TF.add(vgg_model) 
# model_TF.add(Dense(4096, activation='relu')) 
# model.add(Dropout(0.25)) 
# model_TF.add(Dense(2048, activation='relu')) 
# model.add(Dropout(0.25))
# model_TF.add(Dense(1024, activation='relu')) 
#model.add(Dropout(0.25))
model_TF.add(Dense(num_classes, activation='sigmoid'))



model_TF.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
      loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
      metrics=['accuracy',f1_m,precision_m, recall_m])

In [12]:
history_TF = model_TF.fit_generator(train_generator_NEW, 
                              epochs=10,  
                              validation_data=validation_generator_NEW,  
                              verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2023-02-08 02:44:17.572428: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '
2023-02-08 02:44:25.347828: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


452/452 [==============================] - 135s 270ms/step - loss: 1.2355 - accuracy: 0.8919 - f1_m: 0.8914 - precision_m: 0.8968 - recall_m: 0.8863 - val_loss: 0.4965 - val_accuracy: 0.9467 - val_f1_m: 0.9459 - val_precision_m: 0.9505 - val_recall_m: 0.9415
Epoch 2/10
452/452 [==============================] - 78s 172ms/step - loss: 0.6859 - accuracy: 0.9326 - f1_m: 0.9315 - precision_m: 0.9351 - recall_m: 0.9280 - val_loss: 2.2436 - val_accuracy: 0.8604 - val_f1_m: 0.8593 - val_precision_m: 0.8641 - val_recall_m: 0.8547
Epoch 3/10
335/452 [=====================>........] - ETA: 13s - loss: 0.5059 - accuracy: 0.9477 - f1_m: 0.9478 - precision_m: 0.9511 - recall_m: 0.9447

KeyboardInterrupt: 

In [ ]:
plt.plot(history_TF.history['f1_m'])
plt.plot(history_TF.history['val_f1_m'])
plt.title('F1 Score')
plt.ylabel('f1')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss= history_TF.evaluate(test_generator_NEW, verbose=1)

In [ ]:
dense = tf.keras.applications.DenseNet121(include_top=False,  
                                     input_shape=input_shape) 
output = dense.layers[-1].output 
output = keras.layers.Flatten()(output) 
dense_model = Model(dense.input, output) 
dense_model.trainable = True


In [ ]:
model = Sequential() 

model.add(dense_model) 
# model.add(Dense(4096, activation='relu')) 
#model.add(Dropout(0.25)) 
# model.add(Dense(2048, activation='relu')) 
#model.add(Dropout(0.25))
# model.add(Dense(1024, activation='relu')) 
#model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='sigmoid'))



model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
      loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
      metrics=['accuracy',f1_m,precision_m, recall_m])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(train_generator_NEW, 
                              epochs=10,  
                              validation_data=validation_generator_NEW,  
                              verbose=1)

In [ ]:
plt.plot(history.history['f1_m'])
plt.plot(history.history['val_f1_m'])
plt.title('F1 Score')
plt.ylabel('f1')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss= history.evaluate(test_generator_NEW, verbose=1)